In [30]:
import pandas as pd
import datetime
import dateutil.parser
import time
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import ElementClickInterceptedException
from msedge.selenium_tools import EdgeOptions, Edge

def obtener_datos_tweets(contenedor):
    """Extraer datos de los tweets"""
    
    # username
    nombreUsuario = contenedor.find_element_by_xpath("./div[1]/div[1]/div[1]/div[2]/div[2]//span/span").text
    # twitter cuenta
    twitterCuenta = contenedor.find_element_by_xpath(".//span[contains(text(), '@')]").text
    try:
        # Fecha
        fecha = datetime.datetime.strftime(dateutil.parser.parse(
            contenedor.find_element_by_xpath(".//time").get_attribute("datetime")), '%d/%m/%Y %H:%M:%S')
        #fecha = contenedor.find_element_by_xpath(".//time").get_attribute("datetime")
    except NoSuchElementException:
        return
    # Contenido del tweet
    contenido = contenedor.find_element_by_xpath(".//div[1]/div[1]/div[1]/div[2]/div[2]/div[2]/div[1]/div[1]").text
    # Link
    link = contenedor.find_element_by_xpath(".//div[1]/div[1]/div[1]/div[2]/div[2]/div[1]/div[1]/div[1]/div[1]/a"
                                           ).get_attribute("href")
    # Contador respuestas
    replyCount = contenedor.find_element_by_xpath(".//div[@data-testid='reply']").text
    if replyCount == '':
        replyCount = str(0)
    # Retweets
    retweetCount = contenedor.find_element_by_xpath(".//div[@data-testid='retweet']").text
    if retweetCount == '':
        retweetCount = str(0)
    # Likes
    likeCount = contenedor.find_element_by_xpath(".//div[@data-testid='like']").text
    if likeCount == '':
        likeCount = str(0)
    interactions = int(retweetCount) + int(likeCount)
    
    tweet = (fecha, nombreUsuario, twitterCuenta, contenido, link, str(interactions), replyCount)
    return tweet

def iniciar_driver(filtro):
    driver = Edge(options = options)
    driver.get("https://twitter.com/search?q=" + filtro + "&src=typed_query&f=live")
    time.sleep(5)
    return driver

def obtener_tweets(driver):
    """Obtener todos los tweets de la página"""
    
    data = []
    tweet_ids = set()
    i = 0
    last_height = driver.execute_script("return window.pageYOffset;")
    scrolling = True

    while scrolling:
        contenedores = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')
        for contenedor in contenedores:
            tweet = obtener_datos_tweets(contenedor)
            if tweet:
                tweet_id = ''.join(tweet)
                if tweet_id not in tweet_ids:
                    tweet_ids.add(tweet_id)
                    data.append(tweet)
                
        # Scroll hasta llegar al final de la página
        i += 600
        #driver.execute_script('window.scrollTo(0, ' + str(i) + ');')
        driver.execute_script('window.scrollTo({ top: ' + str(i) + ', behavior: "smooth" });')

        # Espera para cargar la página
        time.sleep(2)

        # Calcular la nueva posición del scroll y compararla con la anterior
        new_height = driver.execute_script("return window.pageYOffset;")
        if new_height == last_height:
            break
        last_height = new_height

    driver.close()
    return data

def crear_dataFrame(data):
    dataFrame = pd.DataFrame(data,
                      columns=['Create Time','User Name', 'User', 'Message', 'Link', 'Interactions', 'Replies'],
                      index=None)
    return dataFrame

def exportar(datos1, datos2):
    dataListening = pd.concat([datos1, datos2], axis = 0)
    dataListening = dataListening.reset_index(drop = True)
    dataListening.to_excel("dataListeningTwitter.xlsx")
    return dataListening

# Ingresar la fecha inicial, fecha final y validar las fechas
while True:
    try:
        startTime = input("Ingrese la fecha inicial en formato YYYY-MM-DD: ")
        startTime = datetime.datetime.strptime(startTime, '%Y-%m-%d')
        break
    except:
        print("El formato debe ser YYYY-MM-DD")
startTime = startTime.strftime('%Y-%m-%d')
while True:
    try:
        endTime = input("Ingrese la fecha final en formato YYYY-MM-DD: ")
        endTime = datetime.datetime.strptime(endTime, '%Y-%m-%d')
        break
    except:
        print("El formato debe ser YYYY-MM-DD")
endTime = endTime.strftime('%Y-%m-%d')

# El WebDriver debe estar en la misma carpeta que el Script
options = EdgeOptions()
options.use_chromium = True

# Filtros de búsqueda. El filtro1 busca palabras y el filtro2 frases
filtro1 = "(Unisinú OR unisinu OR Unisinu OR @UnisinuOficial) until:" + endTime + " since:" + startTime + " -filter:replies"
filtro2 = '("Universidad del sinú") until:' + endTime + ' since:' + startTime + ' -filter:replies'

datosFiltro1 = crear_dataFrame(obtener_tweets(iniciar_driver(filtro1)))
datosFiltro2 = crear_dataFrame(obtener_tweets(iniciar_driver(filtro2)))
resultado = exportar(datosFiltro1, datosFiltro2)

print("Completado")

Ingrese la fecha inicial en formato YYYY-MM-DD: 2021-10-22
Ingrese la fecha final en formato YYYY-MM-DD: 2021-10-28
Completado


In [31]:
resultado

,Create Time,User Name,User,Message,Link,Interactions,Replies
0,27/10/2021 23:44:39,Lupe,@Lupe24513076,Va a estar en la Unisinu? Yo quiero irrrr,https://twitter.com/Lupe24513076/status/145350...,2,0
1,27/10/2021 23:20:18,Parlamento Andino,@Parlandino,La presidenta del modelo de Parlamento Andino...,https://twitter.com/Parlandino/status/14535018...,1,1
2,27/10/2021 23:15:54,Parlamento Andino,@Parlandino,Durante la instalación del Parlamento Andino ...,https://twitter.com/Parlandino/status/14535007...,10,1
3,27/10/2021 23:12:17,Parlamento Andino,@Parlandino,En el marco de la ceremonia de juramentación ...,https://twitter.com/Parlandino/status/14534998...,1,1
4,27/10/2021 23:09:17,Parlamento Andino,@Parlandino,Durante la Instalación del Parlamento Andino ...,https://twitter.com/Parlandino/status/14534990...,2,1
5,27/10/2021 23:05:40,Parlamento Andino,@Parlandino,#Enterate - #Hoy se llevó a cabo la posesión ...,https://twitter.com/Parlandino/status/14534981...,2,0
6,27/10/2021 22:47:32,Kevin Mart,@kevinmart17,Como así que hay Rafa Pérez gratuito en la uni...,https://twitter.com/kevinmart17/status/1453493...,2,2
7,27/10/2021 21:36:21,Oficina Gestión Social Montería,@OFGestionSocial,Nuestra Gestora Social \n@antonellavegag\n en ...,https://twitter.com/OFGestionSocial/status/145...,13,2
8,27/10/2021 16:28:10,Invima,@invimacolombia,Hoy en la sede de \n@UnisinuOficial\n en Monte...,https://twitter.com/invimacolombia/status/1453...,4,0
9,27/10/2021 16:26:57,junior Alejandro ruiz paz,@gerioncrisaor,Capacitación Secop II Contraloría de Montería ...,https://twitter.com/gerioncrisaor/status/14533...,3,0
